In [1]:
from data_generators import get_iterator, get_dataset
from classifiers import theme_classifier

In [2]:
import torch
import torch.nn as nn

In [3]:
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [4]:
train_dataset, val_dataset, test_dataset, review_text_FIELD, theme_FIELD = get_dataset()#vectors = 
                                                                                       #GLOVE_EMBEDDING)

In [5]:
batch_size = 32
train_iter = get_iterator(train_dataset, batch_size, train=True, shuffle=False, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=False, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=False, repeat=False)

In [6]:
list_train = list(val_iter)

In [7]:
batch = list_train[600]
x = batch.review_text.transpose(1, 0).int()[:10]
y = batch.theme.int()

for idx in range(x.shape[0]):
    #print(x.shape, y.shape)
    print("{} | {}".format(' '.join([train_dataset.fields['review_text'].vocab.itos[_] for _ in x[idx]]),
         train_dataset.fields['theme'].vocab.itos[y[idx]]))

As usual sad true story ... but great acting | plot
Anything with Jennifer Lawrence is worth your time . | other
You 'll find them in `` Dirty Grandpa . | other
No direction , worthwhile plot , or acting . | plot
although be warned there are some gory scenes . | plot
One of the best musical movie for me . | other
So disappointed in Jake Gyllenhall for this one . | other
It takes much more than a balance to WALK | other
that highlights the greatest of a true thriller . | other
Overall , though , it left me cold . | other


In [8]:
#review_text_FIELD.vocab.vectors.shape, len(review_text_FIELD.vocab.itos)

In [9]:
theme_FIELD.vocab.itos

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']

In [10]:
vocab_size = len(review_text_FIELD.vocab)#.vectors.shape[0]
label_size = len(theme_FIELD.vocab) - 1
emb_dim = 300#review_text_FIELD.vocab.vectors.shape[1]
vectors = train_dataset.fields["review_text"].vocab.vectors
hidden_dim = 500
layers = 2
dropout = .2

label_size

5

In [48]:
class BaseModel(nn.Module):

    def __init__(self, 
                 ninp = vocab_size, 
                 linp = label_size, 
                 emb_dim = emb_dim, 
                 emb_lab = 20,
                 nhid = hidden_dim, 
                 nout = vocab_size, 
                 nlayers = layers, 
                 dropout = dropout, 
                 vectors = vectors,
                 pretrained = False):
        super().__init__()
        
        self.ninp = ninp
        self.linp = linp
        self.emb_dim = emb_dim
        self.emb_lab = emb_lab
        self.nhid = nhid
        self.nout = nout
        self.nlayers = nlayers
        self.drop = nn.Dropout(dropout)

        self.word_embedding = nn.Embedding(ninp, emb_dim)
        self.label_embedding = nn.Embedding(linp, emb_lab)
        
        self.rnn = nn.LSTM(emb_dim + emb_lab, nhid, nlayers, dropout=dropout)
        self.rnn.flatten_parameters()
        self.decoder = nn.Linear(nhid, nout)
        self.softmax = nn.LogSoftmax(dim=-1)

        if pretrained:
            self.encoder.weight.data = vectors
            
    def init_weights(self):
        initrange = .1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, reviews, labels, hidden):
        R = self.word_embedding(reviews)
        L = self.label_embedding(labels)
        L = torch.cat([L.unsqueeze(0)]*R.shape[0])
        X = torch.cat([R, L], -1)
        
        X, hidden = self.rnn(X, hidden)
        X = X.view(X.size(0)*X.size(1), X.size(2))
        
        X = self.decoder(X)
        log_probs = self.softmax(X)
        return log_probs, hidden
      

In [55]:
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [56]:
model = BaseModel().cuda()

In [57]:
# implement PPL
learning_rate = 0.001
criterion = nn.NLLLoss(reduction='sum', 
                       ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [58]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss_e = 0
    
    total_number_of_words = 0
    with torch.no_grad():
        for i, batch in enumerate(data_source):
            if i == 100: break
            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 1:
                data, targets = batch[:-1,:], batch[1:,:]
                number_of_words = data.shape[0]*data.shape[1]
                
                output, hidden = model(data, labels, hidden)
                output_flat = output.contiguous().view(-1, vocab_size)

                total_loss_e += criterion(output_flat, targets.contiguous().view(-1)).data.float()
                total_number_of_words += number_of_words
                hidden = repackage_hidden(hidden)
            
    return total_loss_e.item() / total_number_of_words

In [59]:
vocab_size

30002

In [60]:
import numpy as np
np.exp(evaluate(val_iter))

30083.068177308916